<a href="https://colab.research.google.com/github/barosturk/bert-kurs-oneri/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gerekli Kütüphaneler


In [11]:
import pandas as pd
import numpy as np

Veri Yükleme

In [12]:
df=pd.read_csv('/content/turkish_news_70000.csv',index_col='id',encoding='utf-8')
df.head(3)

,main_image,published,site,text,title,url
id,,,,,,
72337,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:33:00.000+02:00""","""diken.com.tr""","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""http://www.diken.com.tr/yatirim-bankasi-dolar..."
72338,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:21:00.000+02:00""","""diken.com.tr""","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""http://www.diken.com.tr/piar-arastirma-adana-..."
72339,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T12:20:00.000+02:00""","""diken.com.tr""","""Renaissance Capital: Merkez Bankası bu hafta ...","""Renaissance Capital: Merkez Bankası bu hafta ...","""http://www.diken.com.tr/renaissance-capital-m..."


VERİ HAZIRLAMA VE TEMİZLEME

Data frame yapısını bozmadan sadece text sütunu ile çalışmak için

In [16]:
haber_veriSeti = df[["text"]]
haber_veriSeti.head(3)

,text
id,
72337,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9..."
72338,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i..."
72339,"""Renaissance Capital: Merkez Bankası bu hafta ..."


Veri Hazırlama İşlemi İçin bir fonksiyon oluşturacağız.Bunun için öncelikle kütüphaneleri ekleyelim.

In [19]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Değişkenleri tanımlayalım

In [20]:
nok_isaretleri_kumesi=string.punctuation
etkisiz_kelimeler_kumesi=stopwords.words('turkish')
# Etkisiz kelimeler listesine, biz de eklemeler yapabiliriz.
etkisiz_kelimeler_kumesi.extend(['bir','kadar','sonra'])

Fonksiyonumuzu tanımlayalım

In [21]:
def veri_Temizleme(metin):
  metin=metin.lower()
  # verisetindeki yeni satır karakterlerini boşluk karakteriyle değiştirelim.
  metin = metin.replace("\\n", " ")
  #kesme işareti ve sonrasındaki karakterleri kaldıralım,\w+ : Sonrasındaki bir veya daha fazla kelime karakterini (harf, rakam, alt çizgi) eşleştirir.
  metin = re.sub(r"'(\w+)", "", metin)
  #sayıları kaldıralım
  metin = re.sub(r"[0-9]+", "", metin)
  #noktalama işaretlerini kaldıralım
  metin = "".join([x if x not in nok_isaretleri_kumesi else " " for x in metin])
  metin = " ".join([i for i in metin.split() if i not in etkisiz_kelimeler_kumesi])
  metin = " ".join([i for i in metin.split() if len(i) > 1])

  return metin


Metin Temizleme İşlemi

In [22]:
haber_veriSeti["temizlenen_metin"] = haber_veriSeti["text"].apply(veri_Temizleme)


<ipython-input-22-f3882d06dce4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_veriSeti["temizlenen_metin"] = haber_veriSeti["text"].apply(veri_Temizleme)


Eski metni gösterelim

In [23]:
haber_veriSeti.iloc[5].text

'"Şubat ayında ihracat yüzde 3.7 arttı, ithalat yüzde 18.7 azaldı 04/03/2019 12:20\\nTicaret Bakanı Ruhsar Pekcan şubat ayında ihracatın yüzde 3.7 artışla 14 milyar 312 milyon dolar, ithalatın yüzde 18.7 azalışla 16 milyar 161 milyon dolar olarak gerçekleştiğini açıkladı. Fotoğraf: Reuters\\nBakan Pekcan şunları söyledi: “ Eskiden kullandığımız Özel Ticaret Sistemine göre de şubat ayında ihracatımız yüzde 3,5 artışla 13 milyar 603 milyon dolar olarak gerçekleşmiştir. ÖTS’ye göre ithalatımız şubat ayında yüzde 16,6 düşüşle 15 milyar 793 milyon dolar seviyesinde gerçekleşmiştir. ” Reklam"'

Temizlenmiş metni gösterelim

In [24]:
haber_veriSeti.iloc[5].temizlenen_metin

'şubat ayında ihracat yüzde arttı ithalat yüzde azaldı ticaret bakanı ruhsar pekcan şubat ayında ihracatın yüzde artışla milyar milyon dolar ithalatın yüzde azalışla milyar milyon dolar olarak gerçekleştiğini açıkladı fotoğraf reuters bakan pekcan şunları söyledi eskiden kullandığımız özel ticaret sistemine göre şubat ayında ihracatımız yüzde artışla milyar milyon dolar olarak gerçekleşmiştir öts’ye göre ithalatımız şubat ayında yüzde düşüşle milyar milyon dolar seviyesinde gerçekleşmiştir reklam'

Metni Tokenize Etme İşlemi

In [25]:
haber_veriSeti["temizlenen_metin_token"]=haber_veriSeti["temizlenen_metin"].apply( lambda x:x.split())

<ipython-input-25-e231039ef9a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_veriSeti["temizlenen_metin_token"]=haber_veriSeti["temizlenen_metin"].apply( lambda x:x.split())


Token metni gösterelim

In [26]:
haber_veriSeti.head(5)

,text,temizlenen_metin,temizlenen_metin_token
id,,,
72337,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...",yatırım bankası dolar tl üçüncü çeyrekte ’ı gö...,"[yatırım, bankası, dolar, tl, üçüncü, çeyrekte..."
72338,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i...",pi̇ar araştırma adana antalya’da ‘millet ittif...,"[pi̇ar, araştırma, adana, antalya’da, ‘millet,..."
72339,"""Renaissance Capital: Merkez Bankası bu hafta ...",renaissance capital merkez bankası hafta baz p...,"[renaissance, capital, merkez, bankası, hafta,..."
72340,"""Ağrı’daki İYİ Partililerin istifa gerekçesi: ...",ağrı’daki i̇yi̇ partililerin istifa gerekçesi ...,"[ağrı’daki, i̇yi̇, partililerin, istifa, gerek..."
72341,"""Otomobil pazarı iki ayda yarı yarıya eridi 04...",otomobil pazarı iki ayda yarı yarıya eridi oto...,"[otomobil, pazarı, iki, ayda, yarı, yarıya, er..."


-----------------LDA ile Konu Modelleme Eğitimi----------------------------------------

In [1]:
!pip install numpy==1.23.5 --upgrade --force-reinstall



  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1

gensim kütüphanesini install ettikten sonra import edelim

In [27]:
import gensim

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.0 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


Kelime listesi oluşturalım


In [30]:
tokenlastirilmis_metinler = haber_veriSeti["temizlenen_metin_token"]
kelime_listesi=gensim.corpora.Dictionary(tokenlastirilmis_metinler)

Kelime filtresi


In [31]:
#Kelimenin en az 1 belgede geçmesi gerekiyor ve Kelime belgelerin en fazla %70'inde geçmesine izin verilsin
kelime_listesi.filter_extremes(no_below=1,no_above=0.7)

Terimlerin Vektörleştirilmesi- Döküman Terim matrisinin oluşturulması

In [32]:
dokuman_terim_matrisi=[kelime_listesi.doc2bow(terim) for terim in tokenlastirilmis_metinler]

LDA modelini eğitelim

In [38]:
# veri kümesi 10 farklı temaya bölünsün. epoch sayısı 1 olsun yani 1 kez dönsün
lda_model=gensim.models.ldamodel.LdaModel(corpus=dokuman_terim_matrisi,
                                          id2word=kelime_listesi,
                                          num_topics=15,
                                          passes=1)

Oluşturulan soyut konular içerisinde en fazla bulunan terimleri gösterelim

In [39]:
konular=lda_model.print_topics(num_words=7)

for konu in konular:
  print(konu)

(0, '0.009*"olarak" + 0.006*"eğitim" + 0.005*"türkiye" + 0.005*"yeni" + 0.005*"önemli" + 0.004*"olan" + 0.004*"türk"')
(1, '0.084*"gazetesi" + 0.053*"hürriyet" + 0.045*"yayın" + 0.035*"gezi" + 0.030*"mayıs" + 0.029*"lüks" + 0.024*"medya"')
(2, '0.007*"reklam" + 0.006*"şubat" + 0.006*"olan" + 0.006*"bölüm" + 0.006*"son" + 0.004*"yeni" + 0.004*"sosyal"')
(3, '0.012*"tl" + 0.012*"şubat" + 0.011*"nedeniyle" + 0.009*"pakistan" + 0.008*"hindistan" + 0.007*"altın" + 0.007*"hava"')
(4, '0.047*"yorum" + 0.029*"yorumlar" + 0.025*"com" + 0.023*"reklam" + 0.021*"haberturk" + 0.013*"sayfalarında" + 0.012*"şekilde"')
(5, '0.015*"var" + 0.007*"ben" + 0.007*"değil" + 0.006*"yok" + 0.005*"dedi" + 0.005*"bunu" + 0.005*"zaman"')
(6, '0.007*"hapis" + 0.006*"gözaltına" + 0.006*"hakkında" + 0.006*"polis" + 0.006*"tarafından" + 0.005*"ceza" + 0.005*"yıl"')
(7, '0.030*"yüzde" + 0.018*"bin" + 0.015*"milyon" + 0.013*"milyar" + 0.013*"tl" + 0.012*"dolar" + 0.010*"göre"')
(8, '0.017*"belediye" + 0.017*"chp" + 0.0

LDA Konu modellemesi figürsel gösterimi için kullanılan kütüphaneyi yükleyelim

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.0 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [41]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Görselleştirme için hazırlık
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, dokuman_terim_matrisi, kelime_listesi)

# Göster
pyLDAvis.display(vis)